In [ ]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="my_api_key",
)

# Replace placeholders like {{TABLE_SCHEMA}} with real values,
# because the SDK does not support variables.
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=20000,
    temperature=1,
    system=SYSTEM_PROMPT
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": USER_PROMPT 
                }
            ]
        }
    ]
)
print(message.content)